# Costruzioni dataset

In [1]:
import os
import sentencepiece as spm
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom
import spacy

## Variabili

In [13]:
random_number = str(np.random.randint(0, 100000))
flag_shape = True
flag_shape_type = True
flag_size = True

## GEM

In [ ]:
dataset_names = [
    'testset_D2T-1-CFA_WebNLG_CounterFactual',
    'testset_D2T-1-FA_WebNLG_Factual',
    'testset_D2T-1-FI_WebNLG_Fictional',
    'testset_D2T-2-CFA_Wikidata_CounterFactual',
    'testset_D2T-2-FA_Wikidata_Factual',
    'testset_D2T-2-FI_Wikidata_Fictional',
]

for dataset_name in dataset_names:
    path = "..\\..\\D2T-task\\" + dataset_name

    file_names = []
    for file_name in os.listdir(path):
        if os.path.isfile(os.path.join(path, file_name)):
            url = os.path.join(path, file_name)
            print(url)
            tree = ET.parse(url)
            root = tree.getroot()

            shape = ''
            shape_type = ''
            size = 0
            for entry in root.iter('entry'):
                if flag_shape:
                    shape = entry.get('shape')

                if flag_shape_type:
                    shape_type = entry.get('shape_type')
                
                if flag_size:
                    size = entry.get('size')

                mts = []

                for modifiedtripleset in entry.iter('modifiedtripleset'):
                    mtriples = ""

                    triples = []
                    for mtriple in modifiedtripleset.iter('mtriple'):  
                        triple = mtriple.text.split(" | ")
                        triple = (triple[0], triple[1], triple[2])
                        triples.append(triple)

                    triples.sort(key=lambda x: x[1])
                    for triple in triples:
                        mtriples += triple[0] + " " + triple[1] + " " + triple[2] + " "
                                
                    mts.append(mtriples)

                lexs = []
                for lex in entry.iter('lex'):
                    lexs.append(lex.text)

                for mt in mts:
                    for lex in lexs:
                        dataset.append([mt, lex, shape, shape_type, size])

    dataset = np.array(dataset)

    dataset_pd = pd.DataFrame(dataset, columns=['triple', 'sentence', 'shape', 'shape_type', 'size'])
    dataset_pd.to_csv(dataset_name + ".csv", index=False)

## WebNLG

In [7]:
dataset_types = ["train", "dev"] 
triple_numbers = ["1", "2", "3", "4", "5", "6", "7"]
dataset = []

i = 0
max_length = 0
for dataset_type in dataset_types:
    for triple_number in triple_numbers:
        path = "..\\..\\WebNLG-3.0\\original-corpus\\" + dataset_type + "\\" + triple_number + "triples"
        file_names = []
        for file_name in os.listdir(path):
            if os.path.isfile(os.path.join(path, file_name)):
                i += 1
                url = os.path.join(path, file_name)
                tree = ET.parse(url)
                root = tree.getroot()

                shape = ''
                shape_type = ''
                size = 1
                for entry in root.iter('entry'):
                    if flag_shape:
                        shape = entry.get('shape')

                    if flag_shape_type:
                        shape_type = entry.get('shape_type')

                    if flag_size:
                        size = entry.get('size')

                    mts = []
                    
                    for modifiedtripleset in entry.iter('modifiedtripleset'):
                        mtriples = ""

                        triples = []
                        for mtriple in modifiedtripleset.iter('mtriple'):  
                            triple = mtriple.text.split(" | ")
                            triple = (triple[0], triple[1], triple[2])
                            triples.append(triple)

                        triples.sort(key=lambda x: x[1])
                        for triple in triples:
                            mtriples += triple[0] + " " + triple[1] + " " + triple[2] + " "

                        mts.append(mtriples)

                    lexs = []
                    for lex in entry.iter('lex'):
                        lexs.append(lex.text)
                        #get length of lex.text
                        if len(lex.text) > max_length:
                            max_length = len(lex.text)

                    for mt in mts:
                        for lex in lexs:
                            i += 1
                            dataset.append([mt, lex, shape, shape_type, size])

dataset = np.array(dataset)
print(max_length)

dataset_pd = pd.DataFrame(dataset, columns=['triple', 'sentence', 'shape', 'shape_type', 'size'])
dataset_pd.to_csv(random_number + "_" + dataset_type + ".csv", index=False)

444
